In [14]:
import pandas as pd
import numpy as np

In [15]:
almacen = pd.read_excel('Base Almacén - ITBA.xlsx', sheet_name='lx03 04112013')
estanterias_picking = almacen[almacen['Sector'] == 'Picking']['Ubicación']

In [16]:
movimientos = pd.read_csv("Movimientos Pallet Completo.csv", encoding='ISO-8859-1', sep=';')
movimientos = movimientos[~((movimientos['Ubicproc'].isin(estanterias_picking)) | (movimientos['UbicDest'].isin(estanterias_picking)))]

C:\Users\ianda\AppData\Local\Temp\ipykernel_6056\2295009039.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  movimientos = pd.read_csv("Movimientos Pallet Completo.csv", encoding='ISO-8859-1', sep=';')


In [17]:
maestro = pd.read_excel('Mestro de Articulos - ITBA.xlsx', header=2)
mov_info = pd.merge(movimientos, maestro[['Cód. Material', 'Almacenamiento',	'Descr. Nível 1']], left_on='Material', right_on='Cód. Material', how='left')

In [18]:
mov_info = mov_info.sort_values(by='Feconf', ascending=False)
mov_info = mov_info[mov_info['Mvto'].isin(['Recepción', 'Salidas'])]
mov_info = mov_info[mov_info['Almacenamiento'] != 'STICK']
mov_info["UbicGrouping"] = np.where(mov_info["Mvto"] == "Recepción", mov_info["Ubicproc"], mov_info["UbicDest"])
mov_info["id_viaje"] = mov_info.groupby(["Feconf","Usuario","UbicGrouping"]).ngroup()

In [19]:
mov_info_new = mov_info[["Material","Mvto","id_viaje","Hinicio","Feconf","Almacenamiento"]].rename(columns={"Material":"material","Almacenamiento": "almacenamiento","Mvto":"movimiento","Hinicio":"hora","Feconf":"fecha"}).sort_values(["fecha","hora"])
mov_info_new = mov_info_new.dropna()
mov_info_new[:10000].to_csv("movements.csv", index=False)